In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_file_path = '../input/ag-news-classification-dataset/train.csv'
test_file_path = '../input/ag-news-classification-dataset/test.csv'

In [ ]:
data = pd.read_csv(train_file_path)
test = pd.read_csv(test_file_path)

In [ ]:
data.head()

In [ ]:
print(f'train data shape : {data.shape}\ntest data shape : {test.shape}')

In [ ]:
#combining title and description for better model performance and accuracy
x_train = data['Title']+" "+data['Description']
x_test = test['Title']+" "+test['Description']

In [ ]:
x_train

In [ ]:
y_train = data['Class Index'].apply(lambda x:x-1).values
y_test = test['Class Index'].apply(lambda x:x-1).values

In [ ]:
#check for nan values
data.isnull().sum()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [ ]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(np.append(x_train.values,x_test.values))

word_index = tokenizer.word_index
total_words = len(word_index)+1

x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

X_train_final = pad_sequences(x_train_seq,maxlen=100)
X_test_final = pad_sequences(x_test_seq,maxlen=100)

print(f'train datashape : {X_train_final.shape}')
print(f'test datashape : {X_test_final.shape}')

In [ ]:


total_words

In [ ]:
# model building 
model = Sequential([
    layers.Embedding(total_words,128,input_length=100),
    layers.Bidirectional(layers.LSTM(128,return_sequences=True)),
    layers.Bidirectional(layers.LSTM(64,return_sequences=True)),
    layers.GlobalMaxPooling1D(),
    layers.Dense(512,activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(256,activation='relu'),
    layers.Dense(4,activation='softmax'),
    
    
])

In [ ]:
model.compile(optimizer='adam',
             loss = tf.keras.losses.SparseCategoricalCrossentropy(),
             metrics=['accuracy'])

In [ ]:
model.fit(X_train_final,y_train,epochs=10,validation_data=(X_test_final,y_test),verbose=2)